In [1]:
from resnet import resnet_model
import tensorflow as tf
import os
from pycocotools.coco import COCO

import numpy as np
import skimage.io as io
import skimage.transform as transform
import matplotlib.pyplot as plt
import pylab
pylab.rcParams["figure.figsize"] = (8.0, 10.0)

NUM_CLASSES = None

def load_data():
    global NUM_CLASSES
    
    # Load COCO train data
    dataDir = "../Data/coco"
    dataType = "train2017"
    annFile = "{}/annotations/instances_{}.json".format(dataDir,dataType)
    coco = COCO(annFile)

    catIds = coco.getCatIds(supNms=["kitchen"])
    cats = coco.loadCats(catIds)
    NUM_CLASSES = len(cats)

    print("Load images into memory")
    img_data = []
    img_labels = []
    label_to_cat = {}
    print(catIds)
    for label, catId in enumerate(catIds):
        print(catId)
        img_meta_data = coco.loadImgs(coco.getImgIds(catIds=[catId]))
        for meta_data in img_meta_data[:10]:
            img = io.imread(os.path.join(dataDir, dataType, meta_data["file_name"]))
            img = transform.resize(img, (480, 640))
            img = img.astype(np.float32) / 255.0
            img_data.append(img)
            img_labels.append(label)
        label_to_cat[label] = catId
    return np.stack(img_data), np.array(img_labels)

train_data = load_data()

/home/marcus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


loading annotations into memory...
Done (t=10.79s)
creating index...
index created!
Load images into memory
[44, 46, 47, 48, 49, 50, 51]
44


/home/marcus/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


46
47
48
49
50
51


In [2]:
print(train_data[0].shape)
print(train_data[1].shape)

(70, 480, 640, 3)
(70,)


In [3]:
class ImageNetModel(resnet_model.Model):
  def __init__(self, resnet_size, data_format=None, num_classes=NUM_CLASSES,
               version=resnet_model.DEFAULT_VERSION,
               dtype=resnet_model.DEFAULT_DTYPE):
    """
    Args:
      resnet_size: The number of convolutional layers needed in the model.
      data_format: Either 'channels_first' or 'channels_last', specifying which
        data format to use when setting up the model.
      num_classes: The number of output classes needed from the model. This
        enables users to extend the same model to their own datasets.
      version: Integer representing which version of the ResNet network to use.
        See README for details. Valid values: [1, 2]
      dtype: The TensorFlow dtype to use for calculations.

    Raises:
      ValueError: if invalid resnet_size is chosen
    """

    # For bigger models, we want to use "bottleneck" layers
    if resnet_size < 50:
        bottleneck = False
        final_size = 512
    else:
        bottleneck = True
        final_size = 2048

    super(ImageNetModel, self).__init__(
        resnet_size=resnet_size,
        bottleneck=bottleneck,
        num_classes=num_classes,
        num_filters=64,
        kernel_size=7,
        conv_stride=2,
        first_pool_size=3,
        first_pool_stride=2,
        block_sizes=[3, 4, 6, 3],
        block_strides=[1, 2, 2, 2],
        final_size=final_size,
        version=version,
        data_format=data_format,
        dtype=dtype
    )

In [4]:
# Define dataset placeholders
batch_size = tf.placeholder(tf.int64)
x_ph = tf.placeholder(tf.float32, [None, None, None, 3], name="x")
y_ph = tf.placeholder(tf.int64, [None], name="y")
is_training_ph = tf.placeholder(tf.bool, name="is_training")

# Create dataset and iterator
dataset = tf.data.Dataset.from_tensor_slices((x_ph, y_ph)).batch(batch_size).repeat()
itr = dataset.make_initializable_iterator()

# Create ResNet50
resnet50 = ImageNetModel(50)

# Feed x through network
x, y = itr.get_next()
inputs = resnet50(x, True)

axes = [2, 3]# if self.data_format == "channels_first" else [1, 2]
inputs = tf.reduce_mean(inputs, axes, keepdims=True)
inputs = tf.identity(inputs, "final_reduce_mean")

inputs = tf.reshape(inputs, [-1, 2048])
inputs = tf.layers.dense(inputs=inputs, units=NUM_CLASSES)
logits = tf.identity(inputs, "final_dense")

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logits, 1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
MODEL_NAME = "run1"
BATCH_SIZE = 32
EPOCHS = 1

sess = tf.Session()

# Create file writers
train_summary = tf.summary.FileWriter(os.path.join("ms_coco_classifty", MODEL_NAME, "train"), sess.graph)

# Initialize models vars
sess.run(tf.global_variables_initializer())

# Train loop
for epoch in range(EPOCHS):
    print("Epoch {}/{}: Training...".format(epoch + 1, EPOCHS))

    # Feed train data
    sess.run(itr.initializer, feed_dict={ x_ph: train_data[0], y_ph: train_data[1], batch_size: BATCH_SIZE })

    total_loss = 0.0
    total_accuracy = 0.0
    n_batches = len(train_data[0]) // BATCH_SIZE
    for _ in range(n_batches):
        _, loss_value, acc_value = sess.run([train_op, loss, accuracy], feed_dict={ is_training_ph: True })
        total_loss += loss_value
        total_accuracy += acc_value
    #summary = sess.run(merged, feed_dict={total_loss_ph: total_loss, accuracy_ph: total_accuracy / n_batches})
    #train_summary.add_summary(summary, epoch)

print("Train done")

Epoch 1/1: Training...


ResourceExhaustedError: OOM when allocating tensor with shape[32,256,120,160] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: resnet_model/conv2d_2/Conv2D = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](resnet_model/Relu, resnet_model/conv2d_1/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: Mean_1/_25 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_4356_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_a